# 04_alternative_models.ipynb
## Survival Analysis and Discrete Choice Modeling

This notebook explores advanced causal models beyond binary purchase outcomes:
1. **Survival Analysis (Cox Model)**: Models time-to-conversion to understand how ads affect purchase timing
2. **Discrete Choice Modeling**: Models product selection to understand how ads influence which products are chosen

### Key Research Questions:
- Do ad clicks accelerate the purchase decision (survival)?
- Do clicked products have higher probability of being chosen from consideration sets (choice)?

In [1]:
# --- IMPORTS ---
import os
import sys
import json
import warnings
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, Any, Tuple, Optional, List

import pandas as pd
import numpy as np
from tqdm import tqdm

# Statistical imports
from scipy import stats
from scipy.optimize import minimize
import statsmodels.api as sm

warnings.filterwarnings('ignore')

# Initialize logging
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_log = []

def log(message: str):
    """Add message to output log with timestamp"""
    ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{ts}] {message}"
    output_log.append(log_entry)
    print(log_entry)

log("="*80)
log("SURVIVAL ANALYSIS AND DISCRETE CHOICE MODELING")
log("="*80)
log(f"Analysis timestamp: {timestamp}")

[2025-09-23 06:03:41] ================================================================================
[2025-09-23 06:03:41] SURVIVAL ANALYSIS AND DISCRETE CHOICE MODELING
[2025-09-23 06:03:41] ================================================================================
[2025-09-23 06:03:41] Analysis timestamp: 20250923_060341


## Section 1: Load Data

In [2]:
# Load checkpoint data
raw_data_dir = Path("./data/raw")
processed_data_path = Path("./data/user_journey_causal_dataset.parquet")

def load_checkpoint_data() -> Tuple[Dict[str, pd.DataFrame], Dict[str, Any]]:
    """Load the most recent checkpoint data from raw directory."""
    
    if not raw_data_dir.exists():
        raise FileNotFoundError(f"Data directory not found: {raw_data_dir}")
    
    # Find most recent metadata file
    metadata_files = list(raw_data_dir.glob("metadata_*.json"))
    if not metadata_files:
        raise FileNotFoundError("No metadata files found. Please run 01_data_pull.ipynb first.")
    
    latest_metadata = max(metadata_files, key=lambda x: x.stat().st_mtime)
    
    # Load metadata
    with open(latest_metadata, 'r') as f:
        metadata = json.load(f)
    
    extraction_timestamp = metadata['timestamp']
    log(f"Loading data from extraction: {extraction_timestamp}")
    
    # Load all data files
    data = {}
    required_files = [
        'auctions_users', 'auctions_results', 'impressions', 
        'clicks', 'purchases', 'catalog'
    ]
    optional_files = ['hist_purchases', 'hist_impressions', 'hist_clicks']
    
    for file_type in required_files + optional_files:
        pattern = f"{file_type}_{extraction_timestamp}.parquet"
        file_path = raw_data_dir / pattern
        
        if file_path.exists():
            data[file_type] = pd.read_parquet(file_path)
            log(f"  Loaded {file_type}: {len(data[file_type]):,} rows")
        elif file_type in required_files:
            raise FileNotFoundError(f"Required file not found: {file_path}")
        else:
            log(f"  Optional file not found: {file_type}")
    
    return data, metadata

# Load the data
log("\n" + "="*60)
log("LOADING CHECKPOINT DATA")
log("="*60)

try:
    data, metadata = load_checkpoint_data()
    
    # Extract individual dataframes
    auctions_users = data['auctions_users']
    auctions_results = data['auctions_results']
    impressions = data['impressions']
    clicks = data['clicks']
    purchases = data['purchases']
    catalog = data['catalog']
    
    # Historical data (optional)
    hist_purchases = data.get('hist_purchases', pd.DataFrame())
    hist_impressions = data.get('hist_impressions', pd.DataFrame())
    hist_clicks = data.get('hist_clicks', pd.DataFrame())
    
    log("\n✓ Raw data loaded successfully")
    
    # Load processed metrics
    if processed_data_path.exists():
        metrics = pd.read_parquet(processed_data_path)
        log(f"✓ Processed metrics loaded: {len(metrics):,} rows")
    else:
        log("ERROR: Processed metrics not found. Please run 02_analysis.ipynb first.")
        raise FileNotFoundError(f"Processed data not found: {processed_data_path}")
    
except Exception as e:
    log(f"ERROR: Failed to load data: {e}")
    raise

[2025-09-23 06:03:46] 
[2025-09-23 06:03:46] LOADING CHECKPOINT DATA
[2025-09-23 06:03:46] ============================================================
[2025-09-23 06:03:46] Loading data from extraction: 20250923_043038
[2025-09-23 06:03:46]   Loaded auctions_users: 88,690 rows
[2025-09-23 06:03:47]   Loaded auctions_results: 3,410,770 rows
[2025-09-23 06:03:47]   Loaded impressions: 347,741 rows
[2025-09-23 06:03:47]   Loaded clicks: 11,215 rows
[2025-09-23 06:03:47]   Loaded purchases: 1,859 rows
[2025-09-23 06:03:50]   Loaded catalog: 4,961,480 rows
[2025-09-23 06:03:50]   Loaded hist_purchases: 10,332 rows
[2025-09-23 06:03:51]   Loaded hist_impressions: 1,685,675 rows
[2025-09-23 06:03:51]   Loaded hist_clicks: 59,691 rows
[2025-09-23 06:03:51] 
✓ Raw data loaded successfully
[2025-09-23 06:03:51] ✓ Processed metrics loaded: 269,276 rows


## Section 2: Build Event Stream

In [3]:
log("\n" + "="*60)
log("BUILDING EVENT STREAMS")
log("="*60)

# Build all_events dataframe
log("Creating unified event stream...")
events = []

# Add auctions
auctions = auctions_users.copy()
auctions['event_type'] = 'auction'
auctions['event_time'] = pd.to_datetime(auctions['CREATED_AT'])
auctions['PRODUCT_ID'] = None
auctions['VENDOR_ID'] = None
events.append(auctions[['USER_ID', 'AUCTION_ID', 'event_type', 'event_time', 'PRODUCT_ID', 'VENDOR_ID']])

# Add impressions
impressions_evt = impressions.copy()
impressions_evt['event_type'] = 'impression'
impressions_evt['event_time'] = pd.to_datetime(impressions_evt['OCCURRED_AT'])
events.append(impressions_evt[['USER_ID', 'AUCTION_ID', 'event_type', 'event_time', 'PRODUCT_ID', 'VENDOR_ID']])

# Add clicks
clicks_evt = clicks.copy()
clicks_evt['event_type'] = 'click'
clicks_evt['event_time'] = pd.to_datetime(clicks_evt['OCCURRED_AT'])
events.append(clicks_evt[['USER_ID', 'AUCTION_ID', 'event_type', 'event_time', 'PRODUCT_ID', 'VENDOR_ID']])

# Add purchases
purchases_evt = purchases.copy()
purchases_evt['event_type'] = 'purchase'
purchases_evt['event_time'] = pd.to_datetime(purchases_evt['PURCHASED_AT'])
purchases_evt['AUCTION_ID'] = None
purchases_evt['VENDOR_ID'] = None
events.append(purchases_evt[['USER_ID', 'AUCTION_ID', 'event_type', 'event_time', 'PRODUCT_ID', 'VENDOR_ID']])

# Combine and sort
all_events = pd.concat(events, ignore_index=True)
all_events = all_events.sort_values(['USER_ID', 'event_time'])

# Create journey IDs using session gaps
SESSION_GAP_HOURS = metadata.get('session_gap_hours', 2)
all_events['prev_time'] = all_events.groupby('USER_ID')['event_time'].shift()
all_events['time_diff'] = (all_events['event_time'] - all_events['prev_time']).dt.total_seconds() / 3600
all_events['session_break'] = (all_events['time_diff'] >= SESSION_GAP_HOURS) | all_events['time_diff'].isna()
all_events['journey_id'] = all_events.groupby('USER_ID')['session_break'].cumsum()
all_events['journey_id'] = all_events['USER_ID'] + '_' + all_events['journey_id'].astype(str)

log(f"Total events: {len(all_events):,}")
log(f"Unique journeys: {all_events['journey_id'].nunique():,}")

# Build enhanced event stream with metadata
log("\nEnhancing event stream with metadata...")

# Add ranking information
auction_ranks = auctions_results[['AUCTION_ID', 'PRODUCT_ID', 'RANKING']].copy()
auction_ranks = auction_ranks.dropna()
auction_ranks = auction_ranks.groupby(['AUCTION_ID', 'PRODUCT_ID'])['RANKING'].min().reset_index()

all_events_enhanced = all_events.merge(
    auction_ranks, 
    on=['AUCTION_ID', 'PRODUCT_ID'], 
    how='left'
)

# Add catalog information
all_events_enhanced = all_events_enhanced.merge(
    catalog[['PRODUCT_ID', 'BRAND', 'DEPARTMENT_ID', 'PRICE']].drop_duplicates(),
    on='PRODUCT_ID',
    how='left'
)

all_events_enhanced = all_events_enhanced.sort_values(['USER_ID', 'event_time'])
log(f"Enhanced events created: {len(all_events_enhanced):,} records")

[2025-09-23 06:03:51] 
[2025-09-23 06:03:51] BUILDING EVENT STREAMS
[2025-09-23 06:03:51] ============================================================
[2025-09-23 06:03:51] Creating unified event stream...
[2025-09-23 06:03:51] Total events: 449,505
[2025-09-23 06:03:51] Unique journeys: 10,360
[2025-09-23 06:03:51] 
Enhancing event stream with metadata...
[2025-09-23 06:03:58] Enhanced events created: 449,505 records


## Section 3: Helper Functions for Model Building

In [4]:
# Helper functions for data preparation
def _ensure_datetime(df, col):
    """Ensure column is datetime type"""
    if not np.issubdtype(df[col].dtype, np.datetime64):
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

def _safe_numeric(df, cols):
    """Safely convert columns to numeric"""
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

## Section 4: Survival Analysis - Time to Conversion

In [5]:
pip install lifelines scikit-survival

  Using cached lifelines-0.30.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached autograd-1.8.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached autograd_gamma-0.5.0-py3-none-any.whl
  Using cached ecos-2.0.14-cp313-cp313-macosx_14_0_arm64.whl
Using cached lifelines-0.30.0-py3-none-any.whl (349 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 14.1 MB/s  0:00:00
Using cached autograd-1.8.0-py3-none-any.whl (51 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [lifelines]/8 [lifelines]
Note: you may need to restart the kernel to use updated packages.


In [6]:
log("\n" + "="*80)
log("PART 1: SURVIVAL ANALYSIS - TIME TO CONVERSION")
log("="*80)

def build_survival_dataset(all_events: pd.DataFrame, metrics: pd.DataFrame) -> pd.DataFrame:
    """Build journey-level dataset for survival analysis"""
    log("Building survival dataset...")
    
    _ensure_datetime(all_events, 'event_time')
    
    # Get journey times
    journey_times = (all_events.groupby('journey_id')['event_time']
                    .agg(journey_start_time='min', journey_end_time='max')
                    .reset_index())
    
    # Get first purchase time per journey
    purchases = all_events[all_events['event_type'] == 'purchase'].copy()
    first_purchase = (purchases.groupby('journey_id')['event_time']
                      .min().rename('purchase_time').reset_index())
    
    # Merge times
    survival_df = journey_times.merge(first_purchase, on='journey_id', how='left')
    
    # Calculate durations
    survival_df['time_to_purchase_hr'] = (
        (survival_df['purchase_time'] - survival_df['journey_start_time'])
        .dt.total_seconds() / 3600.0
    )
    survival_df['total_journey_hr'] = (
        (survival_df['journey_end_time'] - survival_df['journey_start_time'])
        .dt.total_seconds() / 3600.0
    )
    
    # Duration is time to purchase if purchased, else total journey time
    survival_df['duration'] = survival_df['time_to_purchase_hr'].fillna(survival_df['total_journey_hr'])
    survival_df['event_observed'] = survival_df['purchase_time'].notna().astype(int)
    
    # Add journey-level covariates from metrics
    journey_covariates = (metrics.groupby('journey_id').agg(
        USER_ID=('USER_ID', 'first'),
        total_clicks=('clicks_on_product', 'sum'),
        total_impressions=('impressions_on_product', 'sum'),
        distinct_products_viewed=('PRODUCT_ID', 'nunique'),
        avg_price_viewed=('PRICE', 'mean')
    ).reset_index())
    
    # Add user-level historical features if available
    user_features = None
    if {'USER_ID', 'hist_purchase_count', 'hist_user_ctr'}.issubset(metrics.columns):
        user_features = metrics[['USER_ID', 'hist_purchase_count', 'hist_user_ctr']].drop_duplicates('USER_ID')
    
    # Merge all features
    output = survival_df[['journey_id', 'duration', 'event_observed']].merge(
        journey_covariates, on='journey_id', how='left'
    )
    
    if user_features is not None:
        output = output.merge(user_features, on='USER_ID', how='left')
    
    # Clean data
    output = output.replace([np.inf, -np.inf], np.nan)
    output = output.dropna(subset=['duration'])
    output = output[output['duration'] > 0].copy()
    
    log(f"Survival dataset created: {len(output):,} journeys")
    log(f"  Journeys with conversion: {output['event_observed'].sum():,} ({output['event_observed'].mean():.2%})")
    log(f"  Mean duration: {output['duration'].mean():.2f} hours")
    log(f"  Median duration: {output['duration'].median():.2f} hours")
    
    return output

# Build survival dataset
final_survival_df = build_survival_dataset(all_events, metrics)

[2025-09-23 06:04:18] 
[2025-09-23 06:04:18] PART 1: SURVIVAL ANALYSIS - TIME TO CONVERSION
[2025-09-23 06:04:18] ================================================================================
[2025-09-23 06:04:18] Building survival dataset...
[2025-09-23 06:04:18] Survival dataset created: 8,523 journeys
[2025-09-23 06:04:18]   Journeys with conversion: 816 (9.57%)
[2025-09-23 06:04:18]   Mean duration: 0.64 hours
[2025-09-23 06:04:18]   Median duration: 0.12 hours


In [7]:
# Survival Analysis EDA
log("\n" + "="*60)
log("SURVIVAL ANALYSIS EDA")
log("="*60)

def survival_eda(final_survival_df: pd.DataFrame):
    """Perform exploratory data analysis for survival data"""
    
    try:
        from lifelines import KaplanMeierFitter
        from lifelines.statistics import logrank_test
        
        log("\n1. OVERALL SURVIVAL CURVE (KAPLAN-MEIER)")
        log("-" * 40)
        
        kmf = KaplanMeierFitter()
        kmf.fit(durations=final_survival_df['duration'],
                event_observed=final_survival_df['event_observed'])
        
        # Get survival probabilities at key time points
        time_points = [1, 2, 5, 10, 24]  # hours
        for t in time_points:
            if t <= final_survival_df['duration'].max():
                survival_prob = kmf.survival_function_at_times(t).iloc[0]
                log(f"  Survival at {t}h: {survival_prob:.4f} (conversion: {1-survival_prob:.4f})")
        
        log("\n2. SURVIVAL BY CLICK STATUS")
        log("-" * 40)
        
        # Split by click status
        has_clicks = final_survival_df['total_clicks'].fillna(0) > 0
        df_clicks = final_survival_df[has_clicks]
        df_no_clicks = final_survival_df[~has_clicks]
        
        log(f"  Journeys with clicks: {len(df_clicks):,} ({len(df_clicks)/len(final_survival_df):.1%})")
        log(f"  Journeys without clicks: {len(df_no_clicks):,} ({len(df_no_clicks)/len(final_survival_df):.1%})")
        
        # Fit separate KM curves
        kmf_clicks = KaplanMeierFitter()
        kmf_clicks.fit(df_clicks['duration'], event_observed=df_clicks['event_observed'])
        
        kmf_no_clicks = KaplanMeierFitter()
        kmf_no_clicks.fit(df_no_clicks['duration'], event_observed=df_no_clicks['event_observed'])
        
        log("\n  Median survival times:")
        median_clicks = kmf_clicks.median_survival_time_
        median_no_clicks = kmf_no_clicks.median_survival_time_
        log(f"    With clicks: {median_clicks:.2f} hours")
        log(f"    Without clicks: {median_no_clicks:.2f} hours")
        
        # Log-rank test
        log("\n3. LOG-RANK TEST")
        log("-" * 40)
        
        if len(df_clicks) > 0 and len(df_no_clicks) > 0:
            results = logrank_test(
                df_clicks['duration'], df_no_clicks['duration'],
                event_observed_A=df_clicks['event_observed'],
                event_observed_B=df_no_clicks['event_observed']
            )
            
            log(f"  Test statistic: {results.test_statistic:.4f}")
            log(f"  p-value: {results.p_value:.6f}")
            
            if results.p_value < 0.05:
                log("  ✓ Significant difference between survival curves at 5% level")
            else:
                log("  ✗ No significant difference between survival curves at 5% level")
        
    except ImportError:
        log("WARNING: lifelines package not installed. Skipping Kaplan-Meier analysis.")
        log("Install with: pip install lifelines")
    
    # Basic statistics without lifelines
    log("\n4. BASIC SURVIVAL STATISTICS")
    log("-" * 40)
    
    # Conversion rates by click intensity
    click_bins = pd.qcut(final_survival_df['total_clicks'].fillna(0), 
                         q=[0, 0.25, 0.5, 0.75, 1.0], 
                         duplicates='drop')
    
    conv_by_clicks = final_survival_df.groupby(click_bins, observed=True).agg({
        'event_observed': ['mean', 'count'],
        'duration': 'median'
    })
    
    log("  Conversion by click quartiles:")
    for idx, row in conv_by_clicks.iterrows():
        conv_rate = row[('event_observed', 'mean')]
        count = row[('event_observed', 'count')]
        median_dur = row[('duration', 'median')]
        log(f"    {idx}: {conv_rate:.2%} conversion, {median_dur:.1f}h median (n={count})")

survival_eda(final_survival_df)

[2025-09-23 06:04:21] 
[2025-09-23 06:04:21] SURVIVAL ANALYSIS EDA
[2025-09-23 06:04:21] ============================================================
[2025-09-23 06:04:21] 
1. OVERALL SURVIVAL CURVE (KAPLAN-MEIER)
[2025-09-23 06:04:21] ----------------------------------------
[2025-09-23 06:04:21]   Survival at 1h: 0.8421 (conversion: 0.1579)
[2025-09-23 06:04:21]   Survival at 2h: 0.7468 (conversion: 0.2532)
[2025-09-23 06:04:21]   Survival at 5h: 0.5848 (conversion: 0.4152)
[2025-09-23 06:04:21]   Survival at 10h: 0.4804 (conversion: 0.5196)
[2025-09-23 06:04:21] 
2. SURVIVAL BY CLICK STATUS
[2025-09-23 06:04:21] ----------------------------------------
[2025-09-23 06:04:21]   Journeys with clicks: 3,014 (35.4%)
[2025-09-23 06:04:21]   Journeys without clicks: 5,509 (64.6%)
[2025-09-23 06:04:21] 
  Median survival times:
[2025-09-23 06:04:21]     With clicks: 8.02 hours
[2025-09-23 06:04:21]     Without clicks: 6.47 hours
[2025-09-23 06:04:21] 
3. LOG-RANK TEST
[2025-09-23 06:04:21] 

In [8]:
# Fit Cox Proportional Hazards Model
log("\n" + "="*60)
log("COX PROPORTIONAL HAZARDS MODEL")
log("="*60)

def fit_cox(final_survival_df: pd.DataFrame, covariates: list):
    """Fit Cox proportional hazards model with multiple backend options"""
    
    log("\nFitting Cox model...")
    log(f"Covariates: {', '.join(covariates)}")
    
    # Prepare data
    X = final_survival_df[covariates].copy()
    y_time = final_survival_df['duration'].to_numpy()
    y_event = final_survival_df['event_observed'].astype(bool).to_numpy()
    
    # Clean data
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    
    # Try scikit-survival first
    try:
        from sklearn.preprocessing import StandardScaler
        from sklearn.pipeline import Pipeline
        from sksurv.linear_model import CoxPHSurvivalAnalysis
        from sksurv.util import Surv
        
        log("Using scikit-survival backend...")
        
        X_scaled = X.astype(np.float32)
        surv_y = Surv.from_arrays(event=y_event, time=y_time)
        
        pipe = Pipeline([
            ('scaler', StandardScaler(with_mean=True)),
            ('cox', CoxPHSurvivalAnalysis(alpha=0.0))
        ])
        pipe.fit(X_scaled, surv_y)
        
        # Extract results
        coefs = pd.Series(pipe.named_steps['cox'].coef_, index=X.columns, name='coef')
        hazard_ratios = np.exp(coefs).rename('HR')
        output = pd.concat([coefs, hazard_ratios], axis=1)
        
        log("\n" + "="*40)
        log("COX MODEL RESULTS (scikit-survival)")
        log("="*40)
        log("\n" + output.to_string())
        
        # C-index (concordance)
        c_index = pipe.score(X_scaled, surv_y)
        log(f"\nConcordance index: {c_index:.4f}")
        
        return {'backend': 'scikit-survival', 'model': pipe, 'summary': output, 'c_index': c_index}
        
    except Exception as e:
        log(f"scikit-survival unavailable or failed: {e}")
        log("Falling back to lifelines...")
    
    # Fallback to lifelines
    try:
        from lifelines import CoxPHFitter
        
        log("Using lifelines backend...")
        
        df_fit = final_survival_df[['duration', 'event_observed'] + covariates].copy()
        df_fit = df_fit.replace([np.inf, -np.inf], np.nan).dropna()
        
        cph = CoxPHFitter()
        cph.fit(df_fit, duration_col='duration', event_col='event_observed', robust=True)
        
        # Extract results
        summary = cph.summary[['coef', 'exp(coef)', 'se(coef)', 'z', 'p']]
        summary = summary.rename(columns={'exp(coef)': 'HR'})
        
        log("\n" + "="*40)
        log("COX MODEL RESULTS (lifelines)")
        log("="*40)
        log("\n" + summary.to_string())
        
        # Model statistics
        log("\n" + "="*40)
        log("MODEL FIT STATISTICS")
        log("="*40)
        log(f"  Concordance index: {cph.concordance_index_:.4f}")
        log(f"  Log-likelihood: {cph.log_likelihood_:.4f}")
        log(f"  AIC: {cph.AIC_:.4f}")
        
        # Check proportional hazards assumption
        log("\n" + "="*40)
        log("PROPORTIONAL HAZARDS TEST")
        log("="*40)
        
        try:
            ph_test = cph.check_assumptions(df_fit, p_value_threshold=0.05, show_plots=False)
            log("  Proportional hazards assumption check completed")
            log("  (Violations would be printed above if any)")
        except:
            log("  Could not perform proportional hazards test")
        
        return {'backend': 'lifelines', 'model': cph, 'summary': summary}
        
    except Exception as e:
        log(f"ERROR: Both survival analysis backends failed: {e}")
        return None

# Define covariates
covariates = ['total_clicks', 'total_impressions', 'avg_price_viewed', 'distinct_products_viewed']

# Add historical features if available
if 'hist_purchase_count' in final_survival_df.columns:
    covariates.append('hist_purchase_count')
if 'hist_user_ctr' in final_survival_df.columns:
    covariates.append('hist_user_ctr')

# Filter to valid covariates
valid_covariates = [c for c in covariates if c in final_survival_df.columns]

# Fit the model
cox_results = fit_cox(final_survival_df, valid_covariates)

[2025-09-23 06:04:25] 
[2025-09-23 06:04:25] COX PROPORTIONAL HAZARDS MODEL
[2025-09-23 06:04:25] ============================================================
[2025-09-23 06:04:25] 
Fitting Cox model...
[2025-09-23 06:04:25] Covariates: total_clicks, total_impressions, avg_price_viewed, distinct_products_viewed, hist_purchase_count, hist_user_ctr
[2025-09-23 06:04:25] scikit-survival unavailable or failed: No module named 'sklearn._loss._loss'
[2025-09-23 06:04:25] Falling back to lifelines...
[2025-09-23 06:04:25] Using lifelines backend...
[2025-09-23 06:04:26] 
[2025-09-23 06:04:26] COX MODEL RESULTS (lifelines)
[2025-09-23 06:04:26] ========================================
[2025-09-23 06:04:26] 
                                  coef        HR  se(coef)         z         p
covariate                                                                     
total_clicks              1.645641e-02  1.016593  0.005167  3.184931  0.001448
total_impressions        -5.679060e-05  0.999943  0.00

In [9]:
# Interpret Cox results
if cox_results:
    log("\n" + "="*60)
    log("KEY INTERPRETATIONS - SURVIVAL ANALYSIS")
    log("="*60)
    
    summary = cox_results['summary']
    
    if 'total_clicks' in summary.index:
        hr_clicks = summary.loc['total_clicks', 'HR']
        log(f"\ntotal_clicks Hazard Ratio: {hr_clicks:.4f}")
        
        if hr_clicks > 1:
            pct_increase = (hr_clicks - 1) * 100
            log(f"  → Each additional click increases the hazard of purchase by {pct_increase:.1f}%")
            log(f"  → This means clicks ACCELERATE conversion (shorter time to purchase)")
        elif hr_clicks < 1:
            pct_decrease = (1 - hr_clicks) * 100
            log(f"  → Each additional click decreases the hazard of purchase by {pct_decrease:.1f}%")
            log(f"  → This means clicks DELAY conversion (longer time to purchase)")
        else:
            log(f"  → Clicks have no effect on conversion timing")
    
    if 'total_impressions' in summary.index:
        hr_impr = summary.loc['total_impressions', 'HR']
        log(f"\ntotal_impressions Hazard Ratio: {hr_impr:.4f}")
        
        if hr_impr > 1:
            log(f"  → More impressions associated with faster conversion")
        else:
            log(f"  → More impressions associated with slower conversion")

## Section 5: Discrete Choice Modeling - Product Selection

In [10]:
log("\n" + "="*80)
log("PART 2: DISCRETE CHOICE MODELING - PRODUCT SELECTION")
log("="*80)

def build_choice_dataset(all_events_enhanced: pd.DataFrame,
                        metrics: pd.DataFrame,
                        sample_k: int = None,
                        time_filter: bool = True) -> pd.DataFrame:
    """Build choice sets for conditional logit modeling"""
    
    log("Building choice dataset...")
    _ensure_datetime(all_events_enhanced, 'event_time')
    
    # Get purchases with instance IDs
    purchases_for_choice = (
        all_events_enhanced.query("event_type == 'purchase'")
        .sort_values(['journey_id', 'event_time'])
        .assign(purchase_order=lambda d: d.groupby('journey_id').cumcount() + 1)
        .rename(columns={'PRODUCT_ID': 'purchased_product_id',
                        'event_time': 'purchase_time'})
    )
    purchases_for_choice['choice_instance_id'] = (
        purchases_for_choice['journey_id'].astype(str) + '__' + 
        purchases_for_choice['purchase_order'].astype(str)
    )
    
    log(f"  Found {len(purchases_for_choice):,} purchase instances")
    
    # Get impressions as candidates
    impressions_for_choice = (
        all_events_enhanced.query("event_type == 'impression'")
        .rename(columns={'event_time': 'impression_time'})
    )
    
    # Create candidate sets
    candidates = (
        purchases_for_choice[['journey_id', 'choice_instance_id', 'purchase_time', 'purchased_product_id']]
        .merge(impressions_for_choice[['journey_id', 'PRODUCT_ID', 'impression_time']],
               on='journey_id', how='left')
    )
    
    # Apply time filter: only products impressed before purchase
    if time_filter:
        candidates = candidates[candidates['impression_time'] <= candidates['purchase_time']]
        log("  Applied time filter: only products impressed before purchase")
    
    # Create choice sets
    choice_sets = candidates[['choice_instance_id', 'journey_id', 'PRODUCT_ID']].drop_duplicates()
    
    # Add chosen flag
    choice_df = choice_sets.merge(
        purchases_for_choice[['choice_instance_id', 'purchased_product_id']],
        on='choice_instance_id', how='left'
    )
    choice_df['chosen'] = (choice_df['PRODUCT_ID'] == choice_df['purchased_product_id']).astype(int)
    choice_df = choice_df.drop(columns='purchased_product_id')
    
    # Add product attributes from metrics
    keep_cols = ['journey_id', 'PRODUCT_ID', 'clicks_on_product', 'impressions_on_product', 'PRICE']
    keep_cols += [c for c in ['BRAND', 'DEPARTMENT_ID'] if c in metrics.columns]
    attributes = metrics[keep_cols].drop_duplicates(['journey_id', 'PRODUCT_ID'])
    
    final_choice = choice_df.merge(attributes, on=['journey_id', 'PRODUCT_ID'], how='left')
    
    # Optional negative sampling
    if sample_k is not None:
        log(f"  Sampling up to {sample_k} non-chosen alternatives per choice set")
        
        def _sample(group):
            positives = group[group['chosen'] == 1]
            negatives = group[group['chosen'] == 0]
            if len(negatives) > sample_k:
                negatives = negatives.sample(sample_k, random_state=42)
            return pd.concat([positives, negatives], ignore_index=True)
        
        final_choice = final_choice.groupby('choice_instance_id', group_keys=False).apply(_sample)
    
    # Sanity filters: each choice set must have exactly 1 chosen product
    chosen_counts = final_choice.groupby('choice_instance_id')['chosen'].sum()
    valid_choices = chosen_counts[chosen_counts == 1].index
    final_choice = final_choice[final_choice['choice_instance_id'].isin(valid_choices)]
    
    # Filter: choice sets must have at least 2 alternatives
    set_sizes = final_choice.groupby('choice_instance_id')['PRODUCT_ID'].nunique()
    valid_sets = set_sizes[set_sizes >= 2].index
    final_choice = final_choice[final_choice['choice_instance_id'].isin(valid_sets)].copy()
    
    # Create binary click variable
    final_choice['was_clicked'] = (final_choice['clicks_on_product'].fillna(0) > 0).astype(int)
    
    log(f"Choice dataset created:")
    log(f"  {len(final_choice['choice_instance_id'].unique()):,} choice instances")
    log(f"  {len(final_choice):,} total alternatives")
    log(f"  Average choice set size: {final_choice.groupby('choice_instance_id').size().mean():.1f}")
    
    return final_choice

# Build choice dataset
final_choice_df = build_choice_dataset(
    all_events_enhanced, 
    metrics, 
    sample_k=10,  # Sample up to 10 non-chosen alternatives
    time_filter=True
)

[2025-09-23 05:55:30] 
[2025-09-23 05:55:30] PART 2: DISCRETE CHOICE MODELING - PRODUCT SELECTION
[2025-09-23 05:55:30] ================================================================================
[2025-09-23 05:55:30] Building choice dataset...
[2025-09-23 05:55:30]   Found 1,859 purchase instances
[2025-09-23 05:55:30]   Applied time filter: only products impressed before purchase
[2025-09-23 05:55:30]   Sampling up to 10 non-chosen alternatives per choice set
[2025-09-23 05:55:30] Choice dataset created:
[2025-09-23 05:55:30]   46 choice instances
[2025-09-23 05:55:30]   464 total alternatives
[2025-09-23 05:55:30]   Average choice set size: 10.1


In [11]:
# Choice Model EDA
log("\n" + "="*60)
log("DISCRETE CHOICE EDA")
log("="*60)

def choice_eda(final_choice_df: pd.DataFrame):
    """Exploratory analysis for choice data"""
    
    log("\n1. ATTRIBUTE COMPARISON: CHOSEN VS NOT CHOSEN")
    log("-" * 40)
    
    comparison = final_choice_df.groupby('chosen').agg(
        PRICE_mean=('PRICE', 'mean'),
        PRICE_median=('PRICE', 'median'),
        clicks_mean=('clicks_on_product', 'mean'),
        clicks_sum=('clicks_on_product', 'sum'),
        impressions_mean=('impressions_on_product', 'mean'),
        impressions_sum=('impressions_on_product', 'sum'),
        was_clicked_rate=('was_clicked', 'mean'),
        count=('was_clicked', 'count')
    ).T
    
    log("\n" + comparison.to_string())
    
    # Calculate lift
    if 'was_clicked_rate' in comparison.index:
        click_rate_chosen = comparison.loc['was_clicked_rate', 1]
        click_rate_not_chosen = comparison.loc['was_clicked_rate', 0]
        if click_rate_not_chosen > 0:
            lift = click_rate_chosen / click_rate_not_chosen
            log(f"\n  Click rate lift for chosen products: {lift:.2f}x")
    
    log("\n2. CHOICE SET SIZE DISTRIBUTION")
    log("-" * 40)
    
    set_sizes = final_choice_df.groupby('choice_instance_id')['PRODUCT_ID'].nunique()
    
    log("  Choice set size statistics:")
    log(f"    Mean: {set_sizes.mean():.1f}")
    log(f"    Median: {set_sizes.median():.0f}")
    log(f"    Min: {set_sizes.min()}")
    log(f"    Max: {set_sizes.max()}")
    log(f"    Std: {set_sizes.std():.1f}")
    
    # Distribution of set sizes
    size_dist = set_sizes.value_counts().sort_index().head(10)
    log("\n  Most common choice set sizes:")
    for size, count in size_dist.items():
        log(f"    {size} products: {count} sets ({count/len(set_sizes):.1%})")
    
    log("\n3. PRICE DISTRIBUTION OF PURCHASED PRODUCTS")
    log("-" * 40)
    
    chosen_products = final_choice_df[final_choice_df['chosen'] == 1]['PRICE'].dropna()
    if len(chosen_products) > 0:
        log(f"  Mean price: ${chosen_products.mean():.2f}")
        log(f"  Median price: ${chosen_products.median():.2f}")
        log(f"  25th percentile: ${chosen_products.quantile(0.25):.2f}")
        log(f"  75th percentile: ${chosen_products.quantile(0.75):.2f}")
    
    log("\n4. ATTRIBUTE CORRELATIONS")
    log("-" * 40)
    
    corr_vars = ['PRICE', 'clicks_on_product', 'impressions_on_product']
    corr_matrix = final_choice_df[corr_vars].corr()
    log("\n" + corr_matrix.to_string())
    
    if abs(corr_matrix.values[np.triu_indices_from(corr_matrix.values, 1)]).max() > 0.7:
        log("\n  WARNING: High correlation detected. May cause multicollinearity issues.")

choice_eda(final_choice_df)

[2025-09-23 05:55:30] 
[2025-09-23 05:55:30] DISCRETE CHOICE EDA
[2025-09-23 05:55:30] ============================================================
[2025-09-23 05:55:30] 
1. ATTRIBUTE COMPARISON: CHOSEN VS NOT CHOSEN
[2025-09-23 05:55:30] ----------------------------------------
[2025-09-23 05:55:30] 
chosen                     0          1
PRICE_mean         74.244019  37.695652
PRICE_median       39.000000  29.000000
clicks_mean         0.078947   1.326087
clicks_sum         33.000000  61.000000
impressions_mean    1.361244   1.369565
impressions_sum   569.000000  63.000000
was_clicked_rate    0.062201   0.891304
count             418.000000  46.000000
[2025-09-23 05:55:30] 
  Click rate lift for chosen products: 14.33x
[2025-09-23 05:55:30] 
2. CHOICE SET SIZE DISTRIBUTION
[2025-09-23 05:55:30] ----------------------------------------
[2025-09-23 05:55:30]   Choice set size statistics:
[2025-09-23 05:55:30]     Mean: 10.1
[2025-09-23 05:55:30]     Median: 11
[2025-09-23 05:55:30]   

In [12]:
# Fit Conditional Logit Model
log("\n" + "="*60)
log("CONDITIONAL LOGIT MODEL")
log("="*60)

def fit_conditional_logit(final_choice_df: pd.DataFrame, X_cols=None):
    """Fit conditional logit model with multiple backend options"""
    
    if X_cols is None:
        X_cols = ['PRICE', 'was_clicked', 'impressions_on_product']
    
    log(f"\nFitting conditional logit with features: {', '.join(X_cols)}")
    
    # Prepare data
    df = final_choice_df[['choice_instance_id', 'PRODUCT_ID', 'chosen'] + X_cols].copy()
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    
    # Create integer IDs
    df['obs_id'], _ = pd.factorize(df['choice_instance_id'])
    df['alt_id'], _ = pd.factorize(df['PRODUCT_ID'])
    
    log(f"  {len(df['obs_id'].unique()):,} choice instances")
    log(f"  {len(df):,} total observations")
    
    # Try xlogit first
    try:
        from xlogit import MultinomialLogit
        
        log("Using xlogit backend...")
        
        model = MultinomialLogit()
        model.fit(
            X=df[X_cols].values,
            y=df['chosen'].values.astype(int),
            alt=df['alt_id'].values.astype(int),
            obs=df['obs_id'].values.astype(int),
            verbose=0
        )
        
        summary = pd.DataFrame({
            'coef': model.coeff_,
            'se': np.sqrt(np.diag(model.cov_))
        }, index=X_cols)
        summary['z'] = summary['coef'] / summary['se']
        summary['p'] = 2 * (1 - stats.norm.cdf(abs(summary['z'])))
        
        log("\n" + "="*40)
        log("CONDITIONAL LOGIT RESULTS (xlogit)")
        log("="*40)
        log("\n" + summary.to_string())
        
        return {'backend': 'xlogit', 'model': model, 'summary': summary}
        
    except Exception as e:
        log(f"xlogit unavailable or failed: {e}")
    
    # Try pylogit
    try:
        import pylogit as pl
        
        log("Using pylogit backend...")
        
        # Create specification
        spec = {c: 'all_same' for c in X_cols}
        
        model = pl.create_choice_model(
            data=df,
            alt_id_col='alt_id',
            obs_id_col='obs_id',
            choice_col='chosen',
            specification=spec,
            model_type='MNL'
        )
        
        model.fit_mle(init_vals=None, print_res=False)
        
        summary = pd.DataFrame({
            'coef': model.params,
            'se': np.sqrt(np.diag(model.variance_matrix)),
        }, index=X_cols)
        summary['z'] = summary['coef'] / summary['se']
        summary['p'] = 2 * (1 - stats.norm.cdf(abs(summary['z'])))
        
        log("\n" + "="*40)
        log("CONDITIONAL LOGIT RESULTS (pylogit)")
        log("="*40)
        log("\n" + summary.to_string())
        
        return {'backend': 'pylogit', 'model': model, 'summary': summary}
        
    except Exception as e:
        log(f"pylogit unavailable or failed: {e}")
    
    # Fallback to custom MLE
    log("Using custom MLE implementation...")
    
    X = df[X_cols].to_numpy(dtype=float)
    y = df['chosen'].to_numpy(dtype=int)
    obs = df['obs_id'].to_numpy(dtype=int)
    
    # Group indices
    counts = np.bincount(obs)
    group_idx = np.r_[0, np.cumsum(counts)]
    
    def nll_grad_hess(beta):
        beta = beta.reshape(-1, 1)
        utilities = X @ beta
        log_likelihood = 0.0
        gradient = np.zeros_like(beta)
        hessian = np.zeros((beta.size, beta.size))
        
        for i in range(len(group_idx) - 1):
            start, end = group_idx[i], group_idx[i + 1]
            u_group = utilities[start:end]
            y_group = y[start:end].reshape(-1, 1)
            X_group = X[start:end, :]
            
            # Numerical stability
            max_u = u_group.max()
            exp_u = np.exp(u_group - max_u)
            denom = exp_u.sum()
            probs = exp_u / denom
            
            log_likelihood += float(y_group.T @ (u_group - max_u) - np.log(denom))
            gradient += X_group.T @ (y_group - probs)
            
            W = np.diagflat(probs) - probs @ probs.T
            hessian -= X_group.T @ W @ X_group
        
        return -log_likelihood, -gradient.flatten(), -hessian
    
    def objective(b): return nll_grad_hess(b)[0]
    def jacobian(b): return nll_grad_hess(b)[1]
    def hess(b): return nll_grad_hess(b)[2]
    
    # Optimize
    beta_init = np.zeros(len(X_cols))
    result = minimize(
        objective, beta_init, 
        jac=jacobian, hess=hess, 
        method='trust-ncg',
        options={'gtol': 1e-6, 'maxiter': 200}
    )
    
    # Extract results
    beta_hat = result.x
    cov_matrix = np.linalg.inv(hess(beta_hat))
    se = np.sqrt(np.diag(cov_matrix))
    z_scores = beta_hat / se
    p_values = 2 * (1 - stats.norm.cdf(abs(z_scores)))
    
    summary = pd.DataFrame({
        'coef': beta_hat,
        'se': se,
        'z': z_scores,
        'p': p_values
    }, index=X_cols)
    
    log("\n" + "="*40)
    log("CONDITIONAL LOGIT RESULTS (custom MLE)")
    log("="*40)
    log("\n" + summary.to_string())
    
    log(f"\nLog-likelihood: {-result.fun:.4f}")
    log(f"Convergence: {result.success}")
    log(f"Iterations: {result.nit}")
    
    return {'backend': 'custom', 'model': {'opt': result, 'cov': cov_matrix}, 'summary': summary}

# Define features for choice model
choice_features = ['PRICE', 'was_clicked', 'impressions_on_product']

# Standardize price for better convergence
final_choice_df['PRICE_std'] = (
    (final_choice_df['PRICE'] - final_choice_df['PRICE'].mean()) / 
    final_choice_df['PRICE'].std()
)
choice_features_std = ['PRICE_std', 'was_clicked', 'impressions_on_product']

# Fit the model
clogit_results = fit_conditional_logit(final_choice_df, X_cols=choice_features_std)

[2025-09-23 05:55:30] 
[2025-09-23 05:55:30] CONDITIONAL LOGIT MODEL
[2025-09-23 05:55:30] ============================================================
[2025-09-23 05:55:30] 
Fitting conditional logit with features: PRICE_std, was_clicked, impressions_on_product
[2025-09-23 05:55:30]   46 choice instances
[2025-09-23 05:55:30]   464 total observations
[2025-09-23 05:55:30] xlogit unavailable or failed: No module named 'xlogit'
[2025-09-23 05:55:30] pylogit unavailable or failed: No module named 'pylogit'
[2025-09-23 05:55:30] Using custom MLE implementation...
[2025-09-23 05:55:30] 
[2025-09-23 05:55:30] CONDITIONAL LOGIT RESULTS (custom MLE)
[2025-09-23 05:55:30] ========================================
[2025-09-23 05:55:30] 
                            coef        se         z             p
PRICE_std              -3.582652  1.526925 -2.346318  1.895993e-02
was_clicked             6.012926  1.139593  5.276383  1.317588e-07
impressions_on_product -0.169836  0.459406 -0.369687  7.116159

In [13]:
# Interpret Choice Model Results
if clogit_results:
    log("\n" + "="*60)
    log("KEY INTERPRETATIONS - DISCRETE CHOICE MODEL")
    log("="*60)
    
    summary = clogit_results['summary']
    
    if 'was_clicked' in summary.index:
        coef_click = summary.loc['was_clicked', 'coef']
        p_click = summary.loc['was_clicked', 'p']
        
        log(f"\nwas_clicked coefficient: {coef_click:.4f} (p={p_click:.4f})")
        
        # Calculate odds ratio / probability change
        exp_coef = np.exp(coef_click)
        log(f"  → Clicking a product multiplies its odds of being chosen by {exp_coef:.2f}")
        
        if p_click < 0.05:
            log(f"  → ✓ Effect is statistically significant at 5% level")
            log(f"  → This is STRONG EVIDENCE of causal incrementality")
            log(f"  → Clicks directly influence product choice within consideration sets")
        else:
            log(f"  → ✗ Effect is not statistically significant at 5% level")
    
    if 'PRICE_std' in summary.index:
        coef_price = summary.loc['PRICE_std', 'coef']
        p_price = summary.loc['PRICE_std', 'p']
        
        log(f"\nPRICE_std coefficient: {coef_price:.4f} (p={p_price:.4f})")
        
        if coef_price < 0:
            log(f"  → Higher prices decrease probability of choice (expected)")
        else:
            log(f"  → Higher prices increase probability of choice (luxury effect?)")
    
    if 'impressions_on_product' in summary.index:
        coef_impr = summary.loc['impressions_on_product', 'coef']
        p_impr = summary.loc['impressions_on_product', 'p']
        
        log(f"\nimpressions_on_product coefficient: {coef_impr:.4f} (p={p_impr:.4f})")
        log(f"  → Each additional impression changes choice probability")
        
        if coef_impr > 0:
            log(f"  → More impressions increase choice probability (exposure effect)")
        else:
            log(f"  → More impressions decrease choice probability (fatigue effect)")

[2025-09-23 05:55:30] 
[2025-09-23 05:55:30] KEY INTERPRETATIONS - DISCRETE CHOICE MODEL
[2025-09-23 05:55:30] ============================================================
[2025-09-23 05:55:30] 
was_clicked coefficient: 6.0129 (p=0.0000)
[2025-09-23 05:55:30]   → Clicking a product multiplies its odds of being chosen by 408.68
[2025-09-23 05:55:30]   → ✓ Effect is statistically significant at 5% level
[2025-09-23 05:55:30]   → This is STRONG EVIDENCE of causal incrementality
[2025-09-23 05:55:30]   → Clicks directly influence product choice within consideration sets
[2025-09-23 05:55:30] 
PRICE_std coefficient: -3.5827 (p=0.0190)
[2025-09-23 05:55:30]   → Higher prices decrease probability of choice (expected)
[2025-09-23 05:55:30] 
impressions_on_product coefficient: -0.1698 (p=0.7116)
[2025-09-23 05:55:30]   → Each additional impression changes choice probability
[2025-09-23 05:55:30]   → More impressions decrease choice probability (fatigue effect)


## Section 6: Save Results

In [14]:
# Save all results to text file
log("\n" + "="*80)
log("ANALYSIS COMPLETE")
log("="*80)

# Summary statistics
log("\nFINAL SUMMARY")
log("-" * 60)
log(f"Survival Analysis:")
log(f"  Journeys analyzed: {len(final_survival_df):,}")
log(f"  Conversion rate: {final_survival_df['event_observed'].mean():.2%}")
log(f"  Median survival time: {final_survival_df['duration'].median():.2f} hours")

log(f"\nDiscrete Choice Model:")
log(f"  Choice instances: {len(final_choice_df['choice_instance_id'].unique()):,}")
log(f"  Total alternatives: {len(final_choice_df):,}")
log(f"  Click rate (chosen): {final_choice_df[final_choice_df['chosen']==1]['was_clicked'].mean():.2%}")
log(f"  Click rate (not chosen): {final_choice_df[final_choice_df['chosen']==0]['was_clicked'].mean():.2%}")

# Save to file
output_path = Path("./data") / f"alternative_models_results_{timestamp}.txt"
with open(output_path, 'w') as f:
    f.write('\n'.join(output_log))

log(f"\n✓ Results saved to: {output_path}")
log(f"Total log entries: {len(output_log)}")

print("\n" + "="*80)
print("NOTEBOOK EXECUTION COMPLETE")
print("="*80)

[2025-09-23 05:55:30] 
[2025-09-23 05:55:30] ANALYSIS COMPLETE
[2025-09-23 05:55:30] ================================================================================
[2025-09-23 05:55:30] 
FINAL SUMMARY
[2025-09-23 05:55:30] ------------------------------------------------------------
[2025-09-23 05:55:30] Survival Analysis:
[2025-09-23 05:55:30]   Journeys analyzed: 8,523
[2025-09-23 05:55:30]   Conversion rate: 9.57%
[2025-09-23 05:55:30]   Median survival time: 0.12 hours
[2025-09-23 05:55:30] 
Discrete Choice Model:
[2025-09-23 05:55:30]   Choice instances: 46
[2025-09-23 05:55:30]   Total alternatives: 464
[2025-09-23 05:55:30]   Click rate (chosen): 89.13%
[2025-09-23 05:55:30]   Click rate (not chosen): 6.22%
[2025-09-23 05:55:30] 
✓ Results saved to: data/alternative_models_results_20250923_055513.txt
[2025-09-23 05:55:30] Total log entries: 139

NOTEBOOK EXECUTION COMPLETE
